In [14]:
import tensorflow as tf
import matplotlib.pyplot as plt

DataSet: https://www.kaggle.com/datasets/jehanbhathena/weather-dataset

In [15]:
IMAGE_SIZE = (299, 299)
DATASET_PATH = '../input/weather-dataset/dataset'
SEED = 49

https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory?version=nightly

https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [16]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    batch_size=32,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=SEED,
    validation_split=0.3,
    subset = 'training', # 'validation'

)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    batch_size=32,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=SEED,
    validation_split=0.3,
    subset = 'validation',
)

In [17]:
class_names = train_ds.class_names
class_names

In [18]:
def process(img, lab):
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, lab

train_ds = train_ds.map(process)
val_ds = val_ds.map(process)

In [19]:
plt.figure(figsize=(12, 12))

for x, y in train_ds.skip(3).take(1):
    for i, (img, lab) in enumerate(zip(x, y)):
        if i>=9:
            break
        plt.subplot(3, 3, i+1)
        img = img.numpy()
        lab = lab.numpy()
#         print(img.min(), img.max())
        plt.imshow(img)
        plt.title(class_names[lab])

In [20]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
)

In [21]:
print(len(base_model.inputs))
print(base_model.outputs)

In [22]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.outputs[0])
output = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

In [23]:
model = tf.keras.Model(inputs = base_model.inputs, outputs = output)
model.summary()

In [25]:
base_model.trainable = False

model.compile(
    loss='sparse_categorical_crossentropy',\
    optimizer='adam',
    metrics = ['accuracy']
)

In [26]:
model.fit(train_ds, validation_data=val_ds, epochs=2)

In [27]:
base_model.trainable = True

adam_optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.00001, # 10^-4
)

model.compile(
    loss='sparse_categorical_crossentropy',\
    optimizer=adam_optimizer,
    metrics = ['accuracy']
)

In [28]:
model.fit(train_ds, validation_data=val_ds, epochs=1)

In [29]:
model.save('./model')

# Testing

In [60]:
# url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFhYYGRgaHRweHRocHBkaHhoeHhwaHBocGhoeIy4lIR4sHyEaJzgmKy8xNTU1HSQ7QDs0Py40NTEBDAwMEA8QGhISHDQhGCE0MTE0NDQxNDE0NDE0NDE0NDQxNDQ0ND80ND8xPzE0NDQ/Pz8/MTE0MTExMTExMTExMf/AABEIAL4BCQMBIgACEQEDEQH/xAAbAAACAwEBAQAAAAAAAAAAAAACAwEEBQAGB//EAD0QAAEDAgIHBgUEAgEDBQEAAAEAAhEhMQNBBBJRYXGR8AUigaGx0RMyUsHhBhSS8UJighYjcjM0Q3OyFf/EABgBAQEBAQEAAAAAAAAAAAAAAAABAgME/8QAGhEBAQEBAQEBAAAAAAAAAAAAABEBEiFBAv/aAAwDAQACEQMRAD8AycOANv5/Hqlt+aMhdSwDM38ovA4+iLVAm/8Adl6teUWEZMkUEkq6xkMLyJJJpO32oPFVKCjZoK+0cUTsbuxJmdmWfnkoLEkNJItU1uTanPkqzHSS2ePEbVZ07EIDBEUn8JGpIDi3cNpOStDW4lDJ3cIumHEEU66pySA6DGzoqwxwpz528kQ+NRki9Jnf7D1TdE0oxIpWRSSBcnxKpAVOtMTMKww+E1PBSMtrB057mapAjeLHIhDokyTJ4Cknaq2jY7i2JufK11pdnkaxBoNvD8LK1e0fQiXBxmo1hlwqrbWBg1pPAEgCaUHFc/S2hoOUcrBI0nEaZeJBAABkitxA/CNgxMctAOsC4im1op/ltQ4hlzWTNhOUnNHoLWV19U6swADbaQc0GNpA1nOYO6RMweYTU0WO3VPdqNp2KtjPyFh5lXJJFdnCFVx2t228eKZ4isDWtfupe6evNcWVnqEDTJWxIFKXyTH4WrAuYRgBoBOUwOKU/Ez25/dTRxgZAog4HcqrnBGXw0pEhmJj1iaCi5pyNrpDGomurCRTdUUy9kUjkgDBmeC4OAnPqylSuDT1COoIlAH7l2vzU9U97ptSUh7zJYBAGd7VTGPiJqfsoc0EGDBOeXCETVHFNDIVb4H+w5KXzBNtoGaTrH6fX2TMR58YerE3dXgMvueSYxoBNZAE+OXgl6Q8uJPQ3eAgIAKR4n2W3Q9r9W1J+yZh4g+YxF9nD7JGqZrVWcRhiIrePsfBSgdebiZ22HBPLiGAkAhw7u0RQniqrAJAlWHvJ7obNojcPl5eqqFQ6+y/r7c0RJFfGN5GfWxcHkDVNMzvJsOtqBrYNbFEX9Hfrt1SIN53D3on4bCKX1qeG1VQAflsfIZD15K29+rquE1PkLU2qMrGj4bhOwBWcF8Hb11zVTB0pzAZqOczuV3D0mA0wARy29cFkWDiR8xm/cBjZXwWgdQDvkMGRa4k+25Yj3msmpNeCIPmpPARXdOW9FrR0nEY0gMrtdNyUh2OQZaJsKWNqcJVRz4vYKW4vnU+wTc1Wl8VxFa7YMngEpzYMCDmTs3Df+FOBqyS4mGwQBmbJmM4EFwgSbCfNBUdiGeuugmMIBEJbGSa2/pS10cfstA8Z8muSW4jwAXOP9pTzcefXiqIhS2vDqSnnU+G2B3gamakGwScOgJsDfeKUQdOQXOpZAHiu2EOEJKBjDz6lQ50mAOPDJE6uzdu3oAzNQdrJjXCIlLczf1koYzr0CCy4mPUeilz4GU7lWDyCZUvO3NGQYrWm99yXqDdyCEvg1qPzSqdrHYOX4Uo8k5o7uQ2m3U0Qa5Jp4n7I3vmK5QNwRNAFDa5jb1CuOjsJ0kczvKa95iZ3Dwz5qG4rQKgybbvyhc2oGdKLKCwIAkidvhQDxKZ8YgmLj1PzH7eCQGxQVB6n15LmMm1Ov6C2LRxA8tLve3XqhbjiASJNoyg9QoeKbdn/j+Su0YnWBiLZKIvsY2CAZdXwpX25pLHkESKTQHZmeKbiv1QMnO+alKWB6yTMJlAD57EZGx7RFLSfs33VxjwW0Nv6lJw2aw+UxO6uz8J+P3AGDxP24BTWgsI1tYxHoFGjPAcSRIbW/GY9EOr3aCREmExmHQBxoa8A2fcqMo0hoo4GhAN6gZ04qdD7xlzoGzIDJJfiBx2C/sOSHCk90G11ob2HoYBEuDtYZZRYE+KsaXhzSRDQKZqnouNqVOQoTNG7FGBpEuLnGZtw/sQs1ad8OgoAgxMMRIi3kFz8bWr5DJKxDl4lXNUtjr7UjHvF4z2lP8AlE9Sqznc/utYCw/X7ZqcQ6xIPE/0usN5HIZe/JLxBXy8UAugdZpjDAre/gELGCRn9uPqhf8AMQaicvJA1zobTM/0oIcL34+iHExG2vSR+diDB0gnuwLznzWN3U3TiYynYTwUsxhq0EEG+Z2lKOISe8KCkA33JbpAgn8ZqbtSrDcYSSWzxpwSdJxBHV0pj4Em2/M+G5U34hOZAz62q5gcwkgmYmlRZWNX/dVGUAFp4WTvh7zyVHmWSJd1uUEkmu2Sd/XqmA67iaAA2jl1vXOGtcxHnmT1uVdQRInraVa0XCBmtQJO4Z+NRzSYbqQDw9eZp5JTMQgFp/yNd/4QOfi3LZg0HkBCkFwtw631S34Yhur+FblobIqc52CnqqgLb4HMpzMY2vNTyz3KtjUgRXqAuwHFppU0AG0lRFl+LWTJJy5UHhAWjhvBEkUHUDdNPBZTngGRkIbnXM81YZpJDWsJE3+8cvVEbWGx3zi1TExXePPwTcRoi8mPlzg5lZehaUXO7xz59WVxmkEuIJLQaQBE8TeVn6lA94vUVtagpZdhyTWs+lKDZsVgMcDALSB9VxOwImgQDJBPygAbYr6rRo/2s1JaG0N0trgIAbUzNIp19loDSWxqEkTQ+NE3B0RsZmdtgNwyosqzS6RYjhUGN3FG1hECKGOWfj+FexMBtgYAn8GqViYMkbBfKuRPBWJBNY1tM86zwqlPk18fYJz6DeblA6jUxVV7pStQ5227eCe6lfX7Jbnk26PXotUC/EI4m3XVkgvsdlJ2nNQ9x45eOwI24cio3DdtKWJSy83EqXkARN6lSwNH22qdQZc55KUpbGyCTTJG94aMpPUKNIfUHZfZuCrPw5z3k3ncFPdRIxRE529ylYmkmIBNfRGMP7eG0pT2gGIpl7pAfxp4ZjIpWI6eG1c/DyCBxgE+W/aNwCuZi4IYgaL1Xa53ocNvPZsR6o2jmtKq6S1jHtlt5MW8etir45brOjO3jl1sW3p2itd3iO8KNknL+lgOwodWh2bFloTMOoE8dw90kAyJHzW/tMx2xwt49V5Kx8QANYbifFzr74AEeCBOM8GnCNgogZiEN35eH5ko3sBOqKb8pNvL0Q4gFQydg/8AEe6CMJjnTmjaIoZG/cnAANDRciSa8FXGGSaWyB2DeUQ7UzAEGo3BPxWgMmKu5gfn7b12jOEEE3HL8yVGlsAIkzSAd+QjcpiH9mkjvmkU/wCRt4ASVqYGJPzGNlItdw81lYuIS1jYrWozJNz6eBVjRmPIIDhAoQYmLnmUia1Wva6CZtW1RYCOChuq25IrLdwI65pOBojruZNajaIsIpsVt+GLn5LRM51jcLJiuwgA8F7RqyDNTOwcFpYT2kEibn19FkfFqGx3YgxlXMlamFEDVsoCN1xcB9lOI8TQIHD/AC2bvsrQnGMwY4JbwWiTUZoyc0OK6eHqoEYjjw6+ySx4AnkmYoJgV47kokTS2W9awdq80LiYKkjNLxqcFNxNQTFCo1pOe1KaSTKnEdsz6C1CFufLoQPf16dcUbGxbPNDNYOV/ZVUa1OrKGv2IwJpFSuLN/j6nrYkZAzCLpynPZ1VJxHwbVtwHujLyBuNvSYVc55m07fws4uG6PeeuAT9RuwoMAuA4xG7ei1t/kno0NKZQOHABZWnaKXapFDvoM69bF6DTMOwjOEGNoWstRuPLP0YiJs01PKTxS8Ope85V9h6Ld7T0YtYQ0a0/wCIsKVPG3msFjSBEGtfCyyhb3AReT19/VXNJYA0EXJi+xXtB7Je5ri/ugthu2Z+brIBZWPgajg2Sa2INBnA4IopJApVxjZTKuwD1UPeQ4Bs+HCF2G86xIsaNG439lsdlaEHQX4cXqZGtbLiURT0XQXSCYA2EWWhisbqtpaRXMdeq1jhNtEgZ8Lqp2gwkCBnwpCsIlvZrH6rhSQLeFvCeae3s5lNWhrXceO6isaM3uNA2D0CsBuU7zxyUEYcNpFqAb0GMxpPeAJ6KN7wBGzzO1IPqiIZobBrE5nlwRYfdpkKJjW06ukv3WUzA1mILqDiEwEifL1Qa8Kwg8U8vVLDiTO1ceNkEE24Qgl75nroJBorupDYzz+/tzVbEZAhAplTX+ghe8VJ8NyiCoDKnNE0DGZk9bEBO3w3onXCS90kmY+w91UG99LW9TYBVwNpv6JrhJjJtz6+OSTiP88tgWsXENxDJi2aZjPgBucSdwyCBpDTXj45DkkB2s4k2mT7Io8x1C59CBHW1LL5MlTrzJOagsfEkBFDvqVcPi2xK1wsxI9ZpGkh0d2oMp7dNbHy+ipjRpvZMGjnr0WnSLIZhvyryKpab+ndd7C06sTrG5t3Yy+b1Ke3RnUgGm5aejPxRFJjaFCIZomq1rc2gAnafZeU/VeAQ8aoPywTBgV2+PovaYb8TNgPjCY/R9f52NMf7DPooR4bsHsYYh74IprCmUgD75Zr1rNCAEAWWmzDAFGxllQeyMYaEZH7BZnbWBqMBOZ+y9W3DWN+qMMDDYT9X2KUhWjaKQ0H/UU8EZ0Z48TK2dHwAWM/8W+gTv2wupV5ebdorjUpjMAjJeg/btXHABUqR5t+CdhUHRyYpvXov24XHRgqR5XEwCIgIH4TjAA/K9UdGCW7RwEpHmH4VduZPspbQW6/tbz9HbuSzo7USMHXM+GaEmknltK2n6MyIi/pkkv0VvJIkYTAbnO3uuLaTP4HuVrnQ2qrjaINhVSMx7dnX4SXM6+61Tolc0rF0Wl1KRkOxKQM77wLeaBjCTwzWi7s87Qhfojg2BEm/Ba6xWViGSePRU6sCPE7grjdAcKnrYlv0Z+TTJ4J1iKj6mnXBEIzp17KXaM6bFQcFxNqdSVN/QMOkTF/IIdfd6LnzYCnC6XqO2eSzUj6C8Cltikhs3bzGa+Z6TpbyZk7YnknDHfIaDQAeOZqtV1fScF4dZwPA2VgAbV82wcR7DrMcRGc7T/Z5rc0D9QuAGu3WvBtPEJUr2bSBmmtxAF5jC7ft3LznlyT8H9RTX4Y/l+FG8ej1wiBCwP+owI/7d/9vwuP6mAqMMfy/CEegaQsT9WVYwf7cu6ZSmfqiZJwwP8Alny8VR7R7YOM2rQADI2zCEer0HEDsNhBuxvoFZBXgMDtB7GQHugRA2bAOs1bb2niD/N0n1zQmvaEqZXl8D9SxR7daBdsAnJPd+pWfQ6u8LJNb5XFef8A+pWX1DHELnfqZn0O5hCa3Cgc1UtH7XY4SZbxVoaQ02IK0iHYQSzgBOdjNzKX8dn1IhbsEJbsAJ3xGH/IJZe36gqEuwAEjEwRsV0kWlAYOzmgoPwAlPwhsWmWiyB7RZVGOWRl+VXcwrZeGzVKcGJBkuwzvSXMK2XNYkYmG1TnEZD5QPbFKLSxMJoVV+GPdTnEigcNB8I9SrxYOSDVTnEjzYYDEmAN19/qnsoC6DW278+yEtcZIZ5GmxMdhv1QADGfFJrrB/DLtVgGwuOwmldwFOanSMOXBrCIAgH1KJmjv1bHvb/LlT/kULtGfSSOF0hMWGUN5EVA8lawnwB1VJwdANO8NpvXZlb3TBojpq/lO2virF8wZxJNayjc4HgBz6qmN0FpI759Lq2Oz2UBLqCL/hIVnF9BHXQRtIJAPFarNFwhFOv6VjCw8If4DxSFYZeCRQ7R9k0scSYaSbW5rda9gNGAeA9k34zqRA2+yROmCdAfYMcbE0zTGdk4hEuAnea8hK3H4pNyEOrS/ukOtZI7LMQ416tPVEbdA1TNDGZqeVlqMwvymOwkS6z8PRnG9+KsswyKUjxTrZIS+Kkcwg4tOWW/nKqYriDciaTxuVZZiAZGSlvdrHI+CqEQ7I5cUzCwXEA5cE9jCSO6OITibCwGSCn8N1pUHFhWnNzAqaeCqvbOUKiPjwLmVzMWc1Dt8JYbcqAnvmqS4o3ujxRFo8UCH8fFIxC7bRWcXD3pTlUUnkk3PsueaXVl3BJe0TanX5UIrlDJ2hP1BN1Hwxv80SK37EZnzTHaG0EDZZV8PGNzvp6QnYeIda/X4Wm1z4bTTZTwz+6FrRIA8EOGJoOM7sh6ImPvfiVkPa3d6IiwUpVLY2iJrDeVAQpWFDJJslkmwzpRTItWOBQOMCpmPt+UetUDO/4SGVlNDayN/Xqgcx0n3T2PBSGGhAzTWWQNY9NYUpic1UGwJiEIwoIhSGKWgIgQgF2GCoGAAM6owpQKZhRWSojKtbppchQJxWWg/wBIHC2xOc6fFKJ3zyVSEkVkwUt52Kw50R/SW+legqEPExuSQ3qftwVkHq3mlvcNyQLeSgB2pkBA4DagW5oA3myTiMFrJj3XhLBKBEVup1ztRlwQ/E3IyzMHEkVFcveU0CKxuVPDxWaoOsNpEzTZuT/jSIJg352n1RtfZi6rRAqeuuKMP3eGxUWYgLh3qigkqz8Ws7t/NZFlmJNhMZIi8gdeqpDHEkDO/sEfxQaTfgguNfSbFdr7Otnv4pOu2xIooZixNfOnWSC0AIjw8TEqWAVIJGQ90rDxJuR4qXPFACEDmMJNCntJ6oq7HkZmqa13FBYY47E5r0nCPJN19yA2P3JmvvhJD2jZwXAj+qoLAeic8AKvJNgD5KWNM5hBYcR1CloVcTKKvX4QNJQvbOaW7y8VAPVY9UBO480Ibv8AIqANhtv/ACgMH/KFoS7x5IX4YFJO+VDQAZ1ktzQcwUAgyboMRvD0RPYOvtsS6bjsRAv3JJhNLRvHCnollm/nfmaoBe4W5pD3bEx7IKU50UKJpDsXclfuTsHko0l8C48VnfuP9m8x7oj692Jo7Do+ASxtcLD/AMR9DVf/AG7Ppb/EKt2H/wC2wP8A6sP/APDVfXjeon9uz6W/xC79u36G/wAQnLkCf27fob/ELv27fob/ABCcuQJ/bt+hv8Qu/bs+lv8AEJy5Ar9uz6RyC74LfpbyCauQZOkdpYLS0HVguc0uijS0OcZMf6nhmpxu0sBpaJBLzA1W61g8yYFu48cQuxuxsN5frFx1pmoAGs1zTAA2ONTW1aBRh9h4bXBwL5DpFRA/9U6opb/uYm/vXoIDsPtTRy0O1mgFodDhBAMRIi9RTeETu0sAFokd4ubOqdUFgLnaxiBEHxB2FLw+wcJpkSTDASdWTqluqS6JoGtF4gWmqPE7Gw3F2sXHWJJEgAy0sIgCxBNb2rRBGP2pgNYXgtdAJgCtKGZHd8YVh2k4Qa1xLYeYaY+Y1NIFaAmdglVT2HhnWkvPxBGJJB+ILDXERQUpFLynf/zGwwBzgGRqQR3QZGqKWgxWbDMSgg9qaOP/AJGdRW1qitqqxhYuG4NLS0h0ltqxeBuVPD7Dwmme8Tq6tTZjS0taNwimdTJKvaPozWNAaLFxBNSNYlxrxKCjidrYLdbWGrGIMKHANLnFrXDV1iKarpkxQHdKT29gyWhj3OEkANaS5rS4OeBrfKC11DBtAMhOxuxmP1yXYh1nEmrRdvw3AQLFoA20BkGqnF7Hwu8YLS5xJc0gOMt1S2Y+XVptzvVAD+18LVxHMb8QYZh+p8Ond1jRzha3GmRjn9qNaXB2BiN1Q0mmG6jnFrflcYsTWKApmD2JhNtrRIIE0Aa4vDRS2uZrPKibi6A1wxGhzml7gXEasmAAAJBEUGSCse1cIfE7hIw3BroDHVkg90OLmgQauDRSkrV+E36RyCzsXsxr9fWfiS8apMtkAGS0d2IJ3TwWkwUvO8oI+E36RyC74TfpHIJi5Av4TfpbyC74TfpbyCYuQL+C36W8gu+C36W8gmLkCvgt+lvILvgN+lvIJq5Ak6Oz6W/xCj9qz6GfxCeuQf/Z'
# url = 'https://media.istockphoto.com/photos/sky-and-rainbow-background-picture-id916766854?k=20&m=916766854&s=612x612&w=0&h=ExWPQhsmj_z14spvEtrRFbnlXYBddXPPRe8JrOBbyvU='
url = 'https://i.pinimg.com/originals/21/8b/2d/218b2dea0005efbfef1b78901714b30d.jpg'

file_path = tf.keras.utils.get_file(origin=url)
img = tf.keras.utils.load_img(
    file_path,
    target_size=IMAGE_SIZE
)

In [61]:
import numpy as np

img = np.asarray(img)
img, _ = process(img, '')

In [62]:
plt.imshow(img)

In [63]:
img = img.reshape([1, *IMAGE_SIZE, 3])
img.shape

In [64]:
pred = model.predict(img)

In [65]:
print(pred)
print(np.argmax(pred))
print(class_names[np.argmax(pred)])